In [1]:
import nltk
import numpy as np
import pandas as pd
np.set_printoptions(linewidth=120)
import urllib.request
import gzip
import json
import re
from collections import defaultdict
from numpy import array
from scipy.linalg import svd
from numpy import diag
from numpy import dot

In [2]:
url = "https://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Movies_and_TV.json.gz"
filename = "reviews_Movies_and_TV.json.gz"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/

Mounted at /content/drive
/content/drive/My Drive


In [4]:
def parse(path):
    with open(path, 'r') as f:
        for line in f:
            yield json.loads(line)

# Initialize an empty list to store the sentences
para = []
data_path = 'data'

In [5]:
# Loop through the JSON data and extract the sentences
# for review in parse(data_path):
#     para.append(review['reviewText'])

num_reviews_to_read =  10000

for i, review in enumerate(parse(data_path)):
    para.append(review['reviewText'])
    
    # Stop reading the dataset once the desired number of reviews have been processed
    if i >= num_reviews_to_read - 1:
        break

In [6]:
pattern = re.compile(r'(?<![A-Z])[.!?]\s+')

# iterate over each paragraph and split it into sentences
sentences = []
for paragraph in para:
    sentences.extend(re.split(pattern, paragraph))

sentences = [re.sub(r'[^\w\s]+', '', sentence.strip().lower()) for sentence in sentences]

In [7]:
word_freq = defaultdict(int)
for sent in sentences:
    sent = re.sub(r'[^\w\s\.\-]+', '', sent)
    sent = re.sub('([;:.,!?()])', r' \1 ', sent)
    words = sent.strip().split()  # split the sentence into a list of words
    for word in words:
        word_freq[word] += 1

In [8]:
# Create vocabulary list and replace rare words with '<UNK>'
vocab = ['<UNK>']
rare_words = []
new_sentences = []
word_counts = {}


for sent in sentences:
    words = sent.strip().split()  # split the sentence into a list of words
    new_sent = []
    for word in words:
        # remove non-alphanumeric characters from each word
        word = re.sub(r'\W+', '', word)
        if word_freq[word.lower()] < 5:
            new_sent.append('<UNK>')
            rare_words.append(word.lower())
        else:
            new_sent.append(word)
    new_sentences.append(' '.join(new_sent))

# Add counts of rare words to <UNK>
for word in rare_words:
    if word in word_freq:
        if '<UNK>' not in word_counts:
            word_counts['<UNK>'] = 0
        word_counts['<UNK>'] += word_freq[word.lower()]
        del word_freq[word.lower()]
        
# Add remaining words to vocab
# for word in word_freq.keys():
#     if word_freq[word.lower()] >= 5:
#         vocab.append(word.lower())
#         word_counts[word.lower()] = word_freq[word.lower()]
#     else:
#         word_counts['<UNK>'] += word_freq[word.lower()]
        
# # Sort vocab by frequency
# vocab = sorted(vocab, key=lambda x: word_counts[x], reverse=True)
    
# print("Vocabulary size:", len(vocab))


In [9]:
unique_vocab = set()

for sentence in new_sentences:
    words = sentence.split()
    unique_vocab.update(words)

unique_vocab = list(unique_vocab)

vocabulary = sorted(list(vocab))
print(len(unique_vocab))

9944


In [10]:
# numWords = [len(sentence.split()) for sentence in sentences]
# min_win = min(numWords)
# print(min_win)

# hardcoding window size as smallest possible window size for now
# this can always be changed
# window_size = min_win
window_size = 1

In [11]:
# getting list of words that co occur in window_size
co_words = []

# reversed list of co occur words
re_co_words = []

# loop over each sent, split it into list of words, generate co occur pairs
# l loop executes only if the index l is <= to length of the sentence minus one
# This ensures that we don't go out of bounds when generating pairs of words
for sentence in new_sentences:
    words = sentence.split()
    for i in range(len(words)):
        # print(len(words))
        for j in range(1, window_size+1):
            if i+j < len(words):
                co_words.append([words[i], words[i+j]])
                re_co_words.append([words[i+j], words[i]]) # fix: add to re_co_words instead of co_words
co_words.extend(re_co_words)


In [12]:
a=np.zeros((len(unique_vocab),len(unique_vocab)))
df=pd.DataFrame(a,index=unique_vocab,columns=unique_vocab)

In [13]:
for i in co_words:
    df.at[i[0],i[1]]+=1
#Co-occurence matrix
# df

In [14]:
print(df.iloc[:10, :10])

               quot  warsquot  inconsistent  ensues  tyrant  atoned  \
quot            0.0       0.0           0.0     0.0     0.0     0.0   
warsquot        0.0       0.0           0.0     0.0     0.0     0.0   
inconsistent    0.0       0.0           0.0     0.0     0.0     0.0   
ensues          0.0       0.0           0.0     0.0     0.0     0.0   
tyrant          0.0       0.0           0.0     0.0     0.0     0.0   
atoned          0.0       0.0           0.0     0.0     0.0     0.0   
mayflower       0.0       0.0           0.0     0.0     0.0     0.0   
walks           0.0       0.0           0.0     0.0     0.0     0.0   
realistically   0.0       0.0           0.0     0.0     0.0     0.0   
american        0.0       0.0           0.0     0.0     0.0     0.0   

               mayflower  walks  realistically  american  
quot                 0.0    0.0            0.0       0.0  
warsquot             0.0    0.0            0.0       0.0  
inconsistent         0.0    0.0          

In [15]:
# define a matrix
A = array(df)
# SVD
U, s, VT = svd(A)
Sigma = diag(s)
U.shape,Sigma.shape,VT.shape

((9944, 9944), (9944, 9944), (9944, 9944))

In [16]:
# reducing it to a dimensionality of k = 1000
k = 1000
U_k = U[:,:k]
Sigma_k = Sigma[:k,:k]
VT_k = VT[:k,:]
U_k.shape,Sigma_k.shape,VT_k.shape

((9944, 1000), (1000, 1000), (1000, 9944))

In [17]:
U_df=pd.DataFrame(U_k,index=unique_vocab)
U_df

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
quot,-0.000060,-0.000022,0.000046,0.000080,0.000115,0.000134,-0.000066,-0.000196,-0.000100,-0.000020,...,0.004589,0.000286,-0.000963,0.002951,-0.000541,-0.001280,0.000442,-0.004123,-0.003243,-0.003654
warsquot,-0.000068,-0.000031,0.000171,0.000146,0.000099,-0.000162,-0.000007,-0.000278,0.000098,-0.000066,...,-0.013506,-0.000872,-0.002460,0.002898,0.005822,-0.000907,-0.008831,0.003634,-0.005160,0.008463
inconsistent,-0.000043,-0.000042,0.000064,0.000087,-0.000016,0.000096,-0.000085,-0.000141,-0.000036,0.000017,...,0.002395,-0.005695,0.004770,-0.005161,0.006807,-0.007082,-0.002532,0.000119,-0.002638,-0.005871
ensues,-0.000042,-0.000016,0.000025,-0.000003,0.000118,-0.000023,0.000105,-0.000029,0.000054,-0.000121,...,-0.000660,-0.002722,0.000494,0.003150,0.001822,0.000228,-0.000479,0.002263,-0.002385,0.000428
tyrant,-0.000100,0.000065,-0.000091,-0.000023,0.000010,-0.000145,0.000061,-0.000071,0.000119,-0.000070,...,-0.000037,0.000328,0.000175,-0.000334,-0.000434,-0.000087,-0.000312,-0.000297,0.000426,-0.000021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wishes,-0.000266,-0.000155,0.000254,-0.000393,0.000692,-0.000736,-0.000851,0.000560,0.000698,0.001657,...,0.000908,0.001341,-0.000423,-0.001950,0.000722,0.000292,-0.003493,-0.003487,-0.001269,-0.000249
hinkle,-0.000046,0.000025,0.000050,0.000087,-0.000041,-0.000185,-0.000265,0.000112,0.000130,0.000346,...,-0.008955,-0.002346,-0.069188,0.112629,-0.000590,-0.030283,0.094223,-0.004426,0.051928,0.012321
action,-0.001738,-0.000112,-0.001380,0.000763,-0.000591,-0.001884,0.000213,0.000186,0.001747,-0.000689,...,0.010559,-0.013580,0.031377,-0.012276,-0.024503,0.036175,-0.017311,-0.015860,0.051612,0.029216
wrote,-0.000817,0.000711,0.000445,-0.000711,0.000142,-0.001073,-0.000674,0.001009,-0.000478,-0.001152,...,0.006064,0.001573,-0.007983,0.022126,0.000232,-0.001561,-0.015231,0.021976,-0.013569,-0.000125


In [18]:
emb_list={}
c=0
for i in list(U_df.index):
    emb_list[i]=list(U_df.loc[i])
    c=c+1

In [19]:
count = 0
for key in emb_list:
    if count < 5:
        print(key, ':', emb_list[key])
        count += 1
    else:
        break


quot : [-5.9804452125722385e-05, -2.2460791853709527e-05, 4.5577783980094155e-05, 8.0143665446758e-05, 0.00011463206830750194, 0.00013417284343264502, -6.575418089943552e-05, -0.00019593914726212525, -9.988343892017304e-05, -1.9807104109818147e-05, -0.00011935357201968422, -0.00012756856594109411, -0.00018498704426078705, -0.0001171027346914405, -4.163288997344021e-06, -2.244968244349138e-05, -8.1561497067624e-05, -0.0002877058446685288, -7.727167937464363e-05, -2.6354329414337088e-05, -3.293465606105453e-05, 4.2929660774662995e-05, 2.4262671787662546e-05, 2.189115520878724e-05, 0.0001290938002714015, -1.4898961578206117e-05, -3.4435614243497406e-05, 1.9462239188219183e-05, -2.2280575424147957e-05, 1.1008719560043345e-05, -5.0839785805219945e-05, 7.242316147770692e-06, -1.1403487118021374e-05, 1.1845364700065197e-05, -1.063643060584098e-05, -8.406681799590435e-05, 0.00014117068980004643, 3.152501862606089e-05, -9.864606129755815e-05, -0.00010514890062573307, -3.655431788532164e-05, -0.

In [20]:
# # Define the filename to store the data
# filename = 'emb_list.txt'

# # Open the file for writing
# with open(filename, 'w') as f:
#     # Write the contents of emb_list to the file
#     for key, value in emb_list.items():
#         f.write(f"{key}: {value}\n")
